# Hilbert-Huang Transform (HHT) Analysis Tutorial

This tutorial demonstrates the **recommended HHT workflow** in **gwexpy** using `TimeSeries.hht()`.
HHT is well-suited for nonlinear and non-stationary signals and provides instantaneous frequency information that is difficult to capture with STFT or wavelets.

HHT consists of two main steps:
1. **Empirical Mode Decomposition (EMD/EEMD)**: Decomposes the signal into Intrinsic Mode Functions (IMFs) and a residual.
2. **Hilbert Spectral Analysis**: Computes instantaneous amplitude and frequency for each IMF.

**Note**: This tutorial requires the `PyEMD` (EMD-signal) package.
```bash
pip install EMD-signal
```


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from gwexpy.timeseries import TimeSeries

# Check PyEMD availability
try:
    import PyEMD

    print(f"{PyEMD.__name__} is installed and ready.")
except ImportError:
    raise ImportError(
        "This tutorial requires 'PyEMD' (EMD-signal). Please run: pip install EMD-signal"
    )


## 1. Creating Simulation Data

We create a synthetic, non-stationary signal:
- A low-frequency sine (5 Hz)
- A chirp (80 → 120 Hz)
- A linear trend


In [ ]:
# Time axis (1 second, 1000 samples)
t = np.linspace(0, 1, 1000)
dt = t[1] - t[0]

# Signal components
s1 = 0.5 * np.sin(2 * np.pi * 5 * t)  # Low frequency
s2 = 1.0 * np.sin(2 * np.pi * 80 * t * (1 + 0.5 * t))  # Chirp (80Hz -> 120Hz)
trend = 2.0 * t  # Linear trend

# Create TimeSeries object
data = TimeSeries(s1 + s2 + trend, dt=dt, unit="V", name="Simulation Data")

# Plot
plot = data.plot(title="Original Simulation Signal")
plot.show()


## 2. Recommended workflow: `TimeSeries.hht()`

`TimeSeries.hht()` runs EMD/EEMD and Hilbert analysis in one step.
We pass:
- `emd_kwargs`: EEMD settings (fewer trials for speed in this tutorial)
- `hilbert_kwargs`: padding + smoothing for stable instantaneous frequency

**Tip**: EEMD is stochastic; set `random_state` for reproducibility.


In [ ]:
emd_kwargs = {
    "eemd_trials": 20,
    "random_state": 42,
    "sift_max_iter": 200,
    "stopping_criterion": 0.2,
}
hilbert_kwargs = {
    "pad": 100,
    "if_smooth": 11,
}

result = data.hht(
    emd_method="eemd",
    emd_kwargs=emd_kwargs,
    hilbert_kwargs=hilbert_kwargs,
    output="dict",
)

imfs = result["imfs"]
plot = imfs.plot(figsize=(10, 10), sharex=True, title="IMFs (EEMD)")
plot.show()

plt.figure(figsize=(10, 4))
for key in list(imfs.keys())[:3]:
    if_ts = result["if"][key]
    plt.plot(if_ts.times.value, if_ts.value, label=key)

plt.xlabel("Time [s]")
plt.ylabel("Instantaneous Frequency [Hz]")
plt.title("Instantaneous Frequency (First 3 IMFs)")
plt.legend()
plt.show()


### Interpretation (IMFs + IF)
The first IMF typically captures the high-frequency chirp, and its instantaneous frequency increases over time.


## 3. Hilbert Spectrum (HHTSpectrogram)

Use `output='spectrogram'` to obtain a time–frequency map. `HHTSpectrogram.plot()` uses log scaling by default.


In [ ]:
spec = data.hht(
    output="spectrogram",
    emd_method="eemd",
    emd_kwargs=emd_kwargs,
    hilbert_kwargs=hilbert_kwargs,
    fmin=0,
    fmax=200,
    n_bins=120,
    weight="ia2",
)

plot = spec.plot(figsize=(10, 5))
plot.show()


## 4. Low-level note (optional)

For fine-grained control or research, you can call the lower-level methods directly:
- `emd()` for decomposition
- `hilbert_analysis()` for IA/IF
- `instantaneous_frequency()` / `envelope()` for individual IMFs

For most analysis, `hht()` is the recommended entry point.


### Tips
- Edge effects are common; consider `hilbert_kwargs={'pad': N}` and ignore boundaries.
- Increase `eemd_trials` for cleaner IMFs at the cost of runtime.
- Use `emd_method='emd'` for deterministic results.


### Summary
`TimeSeries.hht()` provides a concise and reproducible workflow for HHT analysis, producing both IMF diagnostics and a Hilbert spectrum.
